# Setup

In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)


## Data 구조 처리
학습용으로 사용하기 위한 이미지 데이터와 좌푯값 전처리 코드.

CSV 구성 형식
- 이미지 경로 , 관절좌표1, 관절좌표2, ... , 관절좌표14, MET Score

처리 형식
- 전체 데이터세트 이미지 최대 크기와 평균 크기 측정 -> 이미지 크기 조정
  크기 조정 비율에 따른 좌표값 재처리


In [2]:
import numpy as np # Numpy 배열 사용용도
import matplotlib.pyplot as plt # 이미지출력 용도
import pandas as pd # csv 파일을 pandas 프레임으로 호출 용도
import skimage
from skimage import io, transform # 이미지 불러오기 및 저장용도

# 주피터 노트북 상에 이미지를 출력하도록 설정
%matplotlib inline

In [3]:
# dataset CSV 파일을 불러와 pandas 프레임으로 저장한 뒤에 Numpy형식의 행렬로 변환하여 testset 변수에 저장
testset=pd.read_csv("/var/data/MET/dataset.csv",header=None).as_matrix()

testset

array([['/var/data/MET/images/walking/20.bmp', '[17, 146]', '[23, 115]',
        ..., '[40, 39]', '[35, 17]', 0.7],
       ['/var/data/MET/images/walking/19.bmp', '[22, 129]', '[12, 116]',
        ..., '[21, 38]', '[14, 16]', 0.7],
       ['/var/data/MET/images/walking/18.bmp', '[23, 153]', '[21, 129]',
        ..., '[23, 41]', '[20, 21]', 0.7],
       ..., 
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/18.jpg',
        '[-1, -1]', '[-1, -1]', ..., '[175, 61]', '[192, 1]', 1.8],
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/19.jpg',
        '[-1, -1]', '[98, 170]', ..., '[172, 59]', '[201, 8]', 1.8],
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/20.jpg',
        '[-1, -1]', '[-1, -1]', ..., '[194, 55]', '[185, 8]', 1.8]], dtype=object)

In [18]:
# 변수 초기화.
# 모든 x축 크기 총합, 모든 y축 크기 총합, 최대 x축 크기, 최대 y축 크기
allsum_x=allsum_y=max_x=max_y=0

for i in range(len(testset)): # testset 길이만큼 반복 // len(testset) = 100개 데이터
    img=skimage.io.imread(testset[i][0]) # 이미지 불러오기 (이미지 저장위치 불러오기)
    # img.shape의 구조는 "행렬의 rows, columns, channels의 갯수"
    
    if max_x<img.shape[0]: max_x=img.shape[0] # 최대 x축 길이값 저장
    if max_y<img.shape[1]: max_y=img.shape[1] # 최대 y축 길이값 저장
    allsum_x+=img.shape[0] # 모든 이미지의 x축 크기 총합
    allsum_y+=img.shape[1] # 모든 이미지의 y축 크기 총합
avg_x=allsum_x/len(testset) # 전체 이미지의 x축 크기 평균
avg_y=allsum_y/len(testset) # 전체 이미지의 y축 크기 평균
print (avg_x,avg_y,max_x,max_y) # x축 크기 평균, y축 크기 평균, 최대 x축 크기, 최대 y축 크기 출력


221.51 243.43 769 834


평균 : (x,y) =(221.51, 243.43)

이미지 크기를 256 x 256 으로 조정

In [16]:
# 문자열의 좌표값을 정수형 좌표값으로 변환하는 함수
def convert_str_to_int(coors):
    coor=np.zeros([len(coors),14,2],dtype=int) # 변환한 좌표를 저장하기 위한 배열 초기화
    for i in range(len(coors)): # 주어진 갯수 만큼 반복
        for j in range(14): # 좌표 개수 만큼 반복
            a=0 # 컴마 위치 확인용
            # "," 위치 체크
            while True: # 무한 반복문
                if coors[i][j+1][a]=="," : break # 해당 문자가 컴마면 무한 반복문 탈출
                else : a+=1 # 컴마가 아니면 다음 문자로 넘어감
            # ,를 기준으로 각각의 좌표 x y를 나누어 int로 형변환 하여 저장
            coor[i][j]=int(coors[i][j+1][1:a]),int(coors[i][j+1][a+1:-1])
    # 이렇게 반복하여 추출한 좌표값을 반환
    return coor


In [32]:
img=skimage.io.imread(testset[1][0])
print(img.shape)
coor_set=convert_str_to_int(testset)
print(coor_set.shape)
print(coor_set[0][0])
print(coor_set[0][1])
print(coor_set[0][0][0]*(256/img.shape[1]))

(173, 60, 3)
(100, 14, 2)
[ 17 146]
[ 23 115]
72.5333333333


In [17]:
def Preprocess(dataset):
    img_set=np.zeros([len(dataset),256,256,3]) 
    # 이미지를 저장해두기 위한 텐서 배열 초기화 - 이미지 갯수 / x크기 / y크기 / RGB
    
    coor_set=convert_str_to_int(dataset) 
    # 문자열의 좌표값을 정수형 좌표값으로 변환하여 저장
    
    for i in range(len(dataset)): # 주어진 데이터 갯수만큼 반복
        img=skimage.io.imread(dataset[i][0]) # 이미지 불러오기
        
        img_set[i]=skimage.transform.resize(img,(256,256)) 
        # 이미지 크기를 재조정 하여 저장
        
        #이미지 크기 변환을 하였으므로 그에 걸맞게 좌표 위치또한 재조정
        for j in range(14): #주어진 좌표 개수 만큼 반복
            if coor_set[i][j][0]==-1: coor_set[i][j][0]=-1 
                # x 좌표 값이 -1 이면 조정을 해주지 않음
            else: coor_set[i][j][0]=coor_set[i][j][0]*(256/img.shape[1]) 
                # x축 크기의 변환된 비율 만큼 곱해 Scaling 
            if coor_set[i][j][1]==-1: coor_set[i][j][1]=-1 # y 좌표 값이 -1 이면 조정을 해주지 않음
            else: coor_set[i][j][1]=coor_set[i][j][1]*(256/img.shape[0]) # y축 크기의 변환된 비율 만큼 곱해 Scaling 
    
    # 이렇게 변환한 이미지 세트와 좌표 세트를 반환
    return img_set, coor_set

In [33]:
# 전처리 함수의 입력으로 불러온 CSV 파일을 저장한 testset 변수로 입력.
# 변환된 이미지 세트와 좌표 세트를 각각 picpic과 coco에다 저장해줌.
picpic,coco=Preprocess(testset)

/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [34]:
# 각 세트의 크기 확인
print (picpic.shape,coco.shape)

(100, 256, 256, 3) (100, 14, 2)


In [39]:
# 이미지와 좌표를 받아와서 이미에 해당 좌표를 입력
# 잘 변환이 되었는지 확인하는 용도

import cv2 # 그림에 숫자 넣기
from skimage import draw # 그림에 원 그리기

def markJoints(img, joints):  
    circSize=5 # 그려질 원의 크기
    font = cv2.FONT_HERSHEY_SIMPLEX # 글씨체 설정
    for i in range(14):
        x = int(joints[i,0]) # X 좌표
        y = int(joints[i,1]) # Y 좌표
        if x!=-1: # 그림에서 보이지 않는 좌표는 찍지 않도록
            rr, cc = skimage.draw.circle(y, x, circSize) # 원 그리기
            skimage.draw.set_color(img, (rr, cc), (1,0,0)) # 색 설정
            cv2.putText(img, str(i+1), (x,y), font, 0.5, (0.5,0.5,0.5), 2, cv2.LINE_AA) # 그림 ㄱ
    return img


In [40]:
# 전처리된 이미지 저장
%mkdir p_img p_with_coors_img # 현재 디렉토리에 처리된 이미지와 좌표를 찍은 이미지들을 저장할 디렉토리 생성
for i in range(len(picpic)):
    skimage.io.imsave("./p_img/p-img_"+str(i)+".jpg",picpic[i]) # 처리된 이미지 저장
    skimage.io.imsave("./p_with_coors_img/p-img_"+str(i)+".jpg",markJoints(picpic[i],coco[i])) # 좌표 입력 이미지 저장

mkdir: cannot create directory ‘p_img’: File exists
mkdir: cannot create directory ‘p_with_coors_img’: File exists


/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
